In [1]:
#load libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
from datetime import datetime, timedelta
import re, os, ast

In [50]:
raw_data = pd.read_csv("/Users/albk/Documents/Code/Real_Projects/basketball_league_webapp/data/Carleton_Canisius #_2023-07-30.csv")

In [51]:
player_event_df = raw_data.copy()

In [52]:
player_event_df['V-event'] = player_event_df['V-event'].astype(str)

In [53]:
#clean data
players_dict = ast.literal_eval(player_event_df.iloc[0]["Home"])
players_list = players_dict["starters"].copy()
players_list.extend(players_dict["reserves"])
players_list.remove("Team")
event_list = ['made layup','missed layup','Assist','Turnover','defensive rebound','enters the game','goes to the bench','missed 3-pt. jump shot','Foul','Steal','made free throw','missed free throw','made jump shot','made 3-pt. jump shot','missed jump shot','offensive rebound']

In [54]:
players_list

['Marjok Okado',
 'Ben Riley',
 'To Randriasalama',
 'Emanuel Milon',
 'Reginald J.-Seraphin',
 'Aubrey Dorey-Havens',
 'Xavier Spencer',
 'Augustas Brazdeikis',
 'Noah Simpson',
 'Daniel Smith',
 'Nathan Christopher']

In [55]:
def make_swap_uppernames(ls):
    upper_players = []
    for player in ls:
        upper_ls = player.upper().split(" ")
        upper_ls[1:], upper_ls[0] = upper_ls[:-1], upper_ls[-1]
        converted_player = ",".join(upper_ls).replace(".", "")
        upper_players.append(converted_player)
    
    return upper_players

In [56]:
players_list = make_swap_uppernames(players_list)
starters = make_swap_uppernames(players_dict["starters"])

### Simply building a dataframe from main one and getting kinds of event with respect to their player

In [57]:
pattern = "([A-Z]+\W*[A-Z]+,[A-Z]+\W*[A-Z]+)"
player_event_df["H-event"] = player_event_df["H-event"].fillna("No Event")
for index, row in player_event_df.iterrows():
    player = re.search(pattern, row["H-event"])
    if player:
        player = player[0].strip()
        player_event_df.loc[index, "player"] = player
    
    for event in event_list:
        if event in row["H-event"]:
            player_event_df.loc[index, "event"] = event

In [58]:
player_event_df.head(n=20)

,Unnamed: 0,Time,Home,H-event,Score,V-event,Visitor,player,event
0,0,Quarter 1,"{'starters': ['Marjok Okado', 'Ben Riley', 'To Randriasalama', 'Emanuel Milon', 'Reginald J.-Seraphin'], 'reserves': ['Aubrey Dorey-Havens', 'Xavier Spencer', 'Augustas Brazdeikis', 'Noah Simpson', 'Daniel Smith', 'Nathan Christopher', 'Team']}",Quarter 1,Quarter 1,Quarter 1,Quarter 1,NaN,NaN
1,1,09:35,NaN,No Event,0 - 2,"GADSDEN,TJ made jump shot",Canisius,NaN,NaN
2,2,09:15,Carleton,"MILON,EMANUEL made jump shot",2 - 2,nan,NaN,"MILON,EMANUEL",made jump shot
3,3,09:15,Carleton,"Assist by OKADO,MARJOK",2 - 2,nan,NaN,"OKADO,MARJOK",Assist
4,4,09:08,NaN,No Event,2 - 2,"Turnover by MITCHELL,FRANK",Canisius,NaN,NaN
5,5,09:08,Carleton,"Steal by RANDRIASALAMA,TO",2 - 2,nan,NaN,"RANDRIASALAMA,TO",Steal
6,6,09:08,Carleton,"Turnover by RANDRIASALAMA,TO",2 - 2,nan,NaN,"RANDRIASALAMA,TO",Turnover
7,7,08:56,NaN,No Event,2 - 4,"STAVESKIE,TAHJ made layup",Canisius,NaN,NaN
8,8,08:34,Carleton,"RILEY,BEN missed jump shot",2 - 4,nan,NaN,"RILEY,BEN",missed jump shot
9,9,08:34,NaN,No Event,2 - 4,"LONG,XZAVIER defensive rebound",Canisius,NaN,NaN


### Calculating in-game time of each player on each quarter and events count for each player also

In [68]:
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
# dependencies
quarter_indices = list(player_event_df[player_event_df['Time'].str.contains('Quarter')].index)
quarter_indices.append(len(player_event_df) - 1)
quarter_dict = {"player":players_list, "time1":[], "score1":[], 
                "time2":[], "score2":[], "time3":[], "score3":[],
                "time4":[], "score4":[]}
v = list(quarter_dict.keys())
v.remove("player")
reorder_ls = ["lineup"] + v
lineup_quarter_dict = {key:[] for key in reorder_ls}
quarter = 1

in_lineup = []
starter_lineup = starters.copy()


not_changed_list = ["not_changed" for n in range(len(players_list))]
event_num_dict = {"player":[]}
time_dict = {"player":players_list, 
             "seconds":list(np.zeros(len(players_list))),
             "points_conceded":list(np.zeros(len(players_list))),
             "timecache":not_changed_list.copy(),
             "scorecache":not_changed_list.copy()}
             
lineup_time_dict = {k:[] for k in list(time_dict.keys())}
lineup_time_dict["lineup"] = lineup_time_dict.pop("player")
lineup_event_dict = {"lineup":[]}

for ind, row in player_event_df.iterrows():
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
    # calculating in-game time of each player
    # -------------------------------------------
    # each quarter end calculations
    if ind in quarter_indices:
        if ind != 0:
            for player in in_lineup:
                player_ind = time_dict["player"].index(player)
                cached_time = time_dict["timecache"][player_ind]
                cached_score = time_dict["scorecache"][player_ind]
                if cached_time == "not_changed":
                    enter_time = datetime.strptime("10:00", "%M:%S")
                    enter_score_index = quarter_indices[quarter - 1] + 1
                    enter_score = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0])
                else:
                    enter_time = datetime.strptime(cached_time, "%M:%S")
                    enter_score = cached_score
                
                seconds = enter_time - datetime.strptime("00:00", "%M:%S")
                seconds = seconds.total_seconds()

                points = int(player_event_df.iloc[ind - 1]["Score"].split("-")[0]) - enter_score

                time_dict["seconds"][player_ind] += seconds
                time_dict["points_conceded"][player_ind] += points
            
            # ---------------------
            # lineup quarter calculations
            lineup_cached_score = lineup_time_dict["scorecache"][-1]
            if lineup_time_dict["timecache"][-1] == "not_changed":
                lineup_time_dict["seconds"] += 600
                enter_score_index = quarter_indices[quarter - 1] + 1
                enter_score = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0])
            else:
                cached_time = lineup_time_dict["timecache"][-1]
                enter_time = datetime.strptime(cached_time, "%M:%S")
                seconds = enter_time - datetime.strptime("00:00", "%M:%S")
                seconds = seconds.total_seconds()
                lineup_time_dict["seconds"][-1] += seconds
                enter_score = int(lineup_cached_score.split("-")[0])
            
            points = int(player_event_df.iloc[ind - 1]["Score"].split("-")[0]) - enter_score

            lineup_time_dict["seconds"][-1] += seconds
            lineup_time_dict["points_conceded"][-1] += points
            # ---------------------

            quarter_dict["player"] = time_dict["player"]
            quarter_dict[f"time{quarter_indices.index(ind)}"] = time_dict["seconds"]
            quarter_dict[f"score{quarter_indices.index(ind)}"] = time_dict["points_conceded"]
            for key in list(lineup_quarter_dict.keys()):
                if key != "lineup":
                    if int(key[-1]) == quarter:
                        if "time" in key:
                            lineup_quarter_dict[f"time{quarter}"].extend(lineup_time_dict["seconds"])
                        else:    
                            lineup_quarter_dict[f"score{quarter}"].extend(lineup_time_dict["points_conceded"])
                    else:
                        length = len(lineup_time_dict["lineup"])
                        zero_list = list(np.zeros(length))
                        lineup_quarter_dict[key].extend(zero_list)
                else:
                    lineup_quarter_dict[key].extend(lineup_time_dict["lineup"])

            
            quarter += 1
            time_dict["timecache"] = not_changed_list.copy()
            time_dict["scorecache"] = not_changed_list.copy()
            time_dict["seconds"] = list(np.zeros(len(players_list)))
            time_dict["points_conceded"] = list(np.zeros(len(players_list)))
            lineup_time_dict = {key:[] for key in lineup_time_dict}

        in_lineup = starter_lineup.copy()
        lineup_time_dict["lineup"].append(sorted(in_lineup.copy()))
        for key in lineup_time_dict:
            if key != "lineup":
                if "cache" in key:
                    lineup_time_dict[key].append("not_changed")
                else:
                    lineup_time_dict[key].append(0)

        for key in list(lineup_event_dict.keys()):
                if key == "lineup":
                    lineup_event_dict[key].append(sorted(in_lineup))
                else:
                    lineup_event_dict[key].append(0)
        continue
    # -------------------------------------------
    # iterating rows calculation
    if not pd.isna(row["player"]) and row["player"] != np.nan and not pd.isna(row["event"]) and row["event"] != np.nan:
        player_index = time_dict["player"].index(row["player"])
        cached_time = time_dict["timecache"][player_index]
        cached_score = time_dict["scorecache"][player_index]
        if "goes to the bench" in row["event"]:
            in_lineup.remove(row["player"])
            if cached_time == "not_changed":
                enter_time = datetime.strptime("10:00", "%M:%S")
                enter_score_index = quarter_indices[quarter - 1] + 1
                enter_score = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0])
            else:
                enter_time = datetime.strptime(cached_time, "%M:%S")
                enter_score = cached_score
            
            seconds = enter_time - datetime.strptime(row["Time"], "%M:%S")
            seconds = seconds.total_seconds()

            points = int(row["Score"].split("-")[0]) - enter_score

            time_dict["seconds"][player_index] += seconds
            time_dict["points_conceded"][player_index] += points

        elif "enters the game" in row["event"]:
            in_lineup.append(row["player"])
            time_dict["timecache"][player_index] = row["Time"]
            time_dict["scorecache"][player_index] = int(row["Score"].split("-")[0])
        
        # ---------------------
        # lineup iterating rows calculation
        last_lineup = lineup_time_dict["lineup"][-1]
        if sorted(in_lineup) != sorted(last_lineup) and len(in_lineup) == 5:
            cached_time = lineup_time_dict["timecache"][-1]
            if lineup_time_dict["timecache"][-1] == "not_changed":
                enter_time = datetime.strptime("10:00", "%M:%S")
                enter_score = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0])
            else:
                enter_time = datetime.strptime(cached_time, "%M:%S")
                enter_score = int(lineup_time_dict["scorecache"][-1].split("-")[0])

            seconds = enter_time - datetime.strptime(row["Time"], "%M:%S")
            seconds = seconds.total_seconds()

            curr_score = int(row["Score"].split("-")[0])
            points = curr_score - enter_score

            lineup_time_dict["seconds"][-1] += seconds
            lineup_time_dict["points_conceded"][-1] += points

            lineup_time_dict["lineup"].append(sorted(in_lineup.copy()))
            for k in lineup_time_dict:
                if k != "lineup":
                    if "cache" in k:
                        lineup_time_dict[k].append("not_changed")
                    else:
                        lineup_time_dict[k].append(0)

            lineup_time_dict["timecache"].append(row["Time"])
            lineup_time_dict["scorecache"].append(row["Score"])
        # ---------------------
    # -------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
        # filling event_num_dict for meaesuring how many times each event occured
        if row["event"] not in event_num_dict:
            event_num_dict[row["event"]] = list(np.zeros(len(event_num_dict["player"])))
        
        if row["player"] not in event_num_dict["player"]:
            event_num_dict["player"].append(row["player"])
            for key in event_num_dict:
                if key != "player":
                    event_num_dict[key].append(0)
        
        player_index = event_num_dict["player"].index(row["player"])
        event_num_dict[row["event"]][player_index] += 1

        # ---------------------
        # lineup event calculations
        if len(lineup_event_dict["lineup"]) == 0:
            lineup_event_dict["lineup"].append(sorted(in_lineup))

        last_lineup = lineup_event_dict["lineup"][-1]
        new_lineup = sorted(in_lineup.copy())
        if row["event"] not in lineup_event_dict:
            lineup_event_dict[row["event"]] = list(np.zeros(len(lineup_event_dict["lineup"])))
        
        if new_lineup == last_lineup:
            lineup_event_dict[row["event"]][-1] += 1
        elif new_lineup != last_lineup and len(new_lineup) == 5:
            for key in list(lineup_event_dict.keys()):
                if key == "lineup":
                    lineup_event_dict[key].append(new_lineup)
                else:
                    lineup_event_dict[key].append(0)
        # ---------------------
# ------------------------------------------------------------------------------------------------------------------------------------------------------------

time_score_df = pd.DataFrame(quarter_dict)
lineup_time_score_df = pd.DataFrame(lineup_quarter_dict)

columns_ls = []
for i in range(1, 5):
    for sub in ["time", "score"]:
        columns_ls.append((f"quarter{i}", sub))

time_score_df.columns = pd.MultiIndex.from_tuples([("player", "player")] + columns_ls)
lineup_time_score_df.columns = pd.MultiIndex.from_tuples([("lineup", "lineup")] + columns_ls)

In [69]:
event_num_df = pd.DataFrame(event_num_dict)
lineup_event_df = pd.DataFrame(lineup_event_dict)
time_sum_ls = [(f"quarter{i}", "time") for i in range(1, 5)]
score_sum_ls = [(f"quarter{i}", "score") for i in range(1, 5)]
time_score_df[("total", "time")] = time_score_df[time_sum_ls].sum(axis=1)
time_score_df[("total", "score")] = time_score_df[score_sum_ls].sum(axis=1)
lineup_time_score_df[("total", "time")] = lineup_time_score_df[time_sum_ls].sum(axis=1)
lineup_time_score_df[("total", "score")] = lineup_time_score_df[score_sum_ls].sum(axis=1)

In [70]:
lineup_event_df.iloc[:-1] # starter lineup will be added automatically at end of the match so we ignore it cause no event will happen there(try proving it by removing .iloc)

,lineup,made jump shot,Assist,Steal,Turnover,missed jump shot,missed 3-pt. jump shot,Foul,enters the game,goes to the bench,defensive rebound,made layup,made 3-pt. jump shot,missed layup,offensive rebound,made free throw,missed free throw
0,"[J-SERAPHIN,REGINALD, MILON,EMANUEL, OKADO,MARJOK, RANDRIASALAMA,TO, RILEY,BEN]",1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"[J-SERAPHIN,REGINALD, MILON,EMANUEL, OKADO,MARJOK, RILEY,BEN, SPENCER,XAVIER]",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"[BRAZDEIKIS,AUGUSTAS, MILON,EMANUEL, OKADO,MARJOK, RILEY,BEN, SPENCER,XAVIER]",0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"[BRAZDEIKIS,AUGUSTAS, MILON,EMANUEL, OKADO,MARJOK, SMITH,DANIEL, SPENCER,XAVIER]",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, MILON,EMANUEL, OKADO,MARJOK, SMITH,DANIEL]",0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0
5,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH, SMITH,DANIEL]",0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,"[DOREY-HAVENS,AUBREY, J-SERAPHIN,REGINALD, RANDRIASALAMA,TO, SIMPSON,NOAH, SMITH,DANIEL]",0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0
7,"[DOREY-HAVENS,AUBREY, RANDRIASALAMA,TO, RILEY,BEN, SIMPSON,NOAH, SMITH,DANIEL]",0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
8,"[RANDRIASALAMA,TO, RILEY,BEN, SIMPSON,NOAH, SMITH,DANIEL, SPENCER,XAVIER]",1.0,1.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0
9,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH, SMITH,DANIEL]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [126]:
lineup_time_score_df

lineup  \
                                                                                   lineup   
0   [LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
1              [LATIFF,WAZIR, RILEY,BEN, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
2              [OKADO,MARJOK, RILEY,BEN, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
3           [BAILEY,ELLIOT, LATIFF,WAZIR, MEYER,EVAN, OKADO,MARJOK, SERAPH,REGINALD JEAN]   
4      [KNIGHT,JACOB, LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR]   
5           [KNIGHT,JACOB, LATIFF,WAZIR, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
6            [DOLMAN,LUKA, LATIFF,WAZIR, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
7                   [BAILEY,ELLIOT, MEYER,EVAN, OKADO,MARJOK, RILEY,BEN, WARNHOLTZ,AIDEN]   
8   [LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
9                  [MEYER,EVAN, OKADO,MARJOK, RILEY,BEN, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
10         [BAILEY,ELLIOT, LATIFF,WAZIR, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
11   [BAILEY,ELLIOT, LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, WARNHOLTZ,AIDEN]   
12             [OKADO,MARJOK, RILEY,BEN, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
13  [OKADO,MARJOK, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
14      [BAILEY,ELLIOT, KNIGHT,JACOB, LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT]   
15              [BAILEY,ELLIOT, LATIFF,WAZIR, RILEY,BEN, SHEPHARD,GRANT, WARNHOLTZ,AIDEN]   
16                 [BAILEY,ELLIOT, LATIFF,WAZIR, OKADO,MARJOK, RILEY,BEN, SAMAHA,GABRAEL]   
17                     [BAILEY,ELLIOT, LATIFF,WAZIR, MEYER,EVAN, OKADO,MARJOK, RILEY,BEN]   
18  [LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
19           [BAILEY,ELLIOT, LATIFF,WAZIR, OKADO,MARJOK, SHEPHARD,GRANT, WARNHOLTZ,AIDEN]   
20           [BAILEY,ELLIOT, LATIFF,WAZIR, OKADO,MARJOK, SAMAHA,GABRAEL, WARNHOLTZ,AIDEN]   
21         [BAILEY,ELLIOT, LATIFF,WAZIR, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
22              [BAILEY,ELLIOT, LATIFF,WAZIR, RILEY,BEN, SAMAHA,GABRAEL, WARNHOLTZ,AIDEN]   
23              [BAILEY,ELLIOT, LATIFF,WAZIR, RILEY,BEN, SHEPHARD,GRANT, WARNHOLTZ,AIDEN]   
24  [LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
25              [BAILEY,ELLIOT, LATIFF,WAZIR, RILEY,BEN, SHEPHARD,GRANT, WARNHOLTZ,AIDEN]   
26              [BAILEY,ELLIOT, LATIFF,WAZIR, RILEY,BEN, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
27             [LATIFF,WAZIR, RILEY,BEN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
28              [BAILEY,ELLIOT, LATIFF,WAZIR, RILEY,BEN, SHEPHARD,GRANT, WARNHOLTZ,AIDEN]   
29         [BAILEY,ELLIOT, LATIFF,WAZIR, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
30             [LATIFF,WAZIR, RILEY,BEN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   

   quarter1       quarter2       quarter3       quarter4        total        
       time score     time score     time score     time score   time score  
0      75.0   0.0      0.0   0.0      0.0   0.0      0.0   0.0   75.0   0.0  
1      83.0   2.0      0.0   0.0      0.0   0.0      0.0   0.0   83.0   2.0  
2      82.0   3.0      0.0   0.0      0.0   0.0      0.0   0.0   82.0   3.0  
3      79.0   0.0      0.0   0.0      0.0   0.0      0.0   0.0   79.0   0.0  
4      78.0   1.0      0.0   0.0      0.0   0.0      0.0   0.0   78.0   1.0  
5      75.0   0.0      0.0   0.0      0.0   0.0      0.0   0.0   75.0   0.0  
6      89.0   2.0      0.0   0.0      0.0   0.0      0.0   0.0   89.0   2.0  
7      78.0   0.0      0.0   0.0      0.0   0.0      0.0   0.0   78.0   0.0  
8       0.0   0.0      0.0   0.0      0.0   0.0      0.0   0.0    0.0   0.0  
9       0.0   0.0    105.0   2.0      0.0   0.0      0.0   0.0  105.0   2.0  
10      0.0   0.0     35.0   3.0      0.0   0.0      0.0   0.0   35.0   3.0  

In [71]:
event_num_df

,player,made jump shot,Assist,Steal,Turnover,missed jump shot,missed 3-pt. jump shot,Foul,enters the game,goes to the bench,defensive rebound,made layup,made 3-pt. jump shot,missed layup,offensive rebound,made free throw,missed free throw
0,"MILON,EMANUEL",4,0.0,1.0,0.0,1.0,0.0,2.0,2.0,5.0,2.0,0.0,0.0,0.0,2.0,1.0,1.0
1,"OKADO,MARJOK",0,5.0,0.0,3.0,5.0,3.0,2.0,4.0,4.0,1.0,3.0,1.0,1.0,0.0,0.0,0.0
2,"RANDRIASALAMA,TO",1,1.0,1.0,2.0,0.0,0.0,0.0,2.0,5.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0
3,"RILEY,BEN",0,0.0,0.0,2.0,1.0,0.0,2.0,3.0,6.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
4,"J-SERAPHIN,REGINALD",0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,"SPENCER,XAVIER",1,1.0,0.0,2.0,3.0,0.0,4.0,6.0,6.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0
6,"BRAZDEIKIS,AUGUSTAS",0,0.0,1.0,4.0,1.0,1.0,1.0,7.0,3.0,5.0,1.0,4.0,3.0,3.0,4.0,1.0
7,"SMITH,DANIEL",0,1.0,0.0,3.0,1.0,2.0,0.0,5.0,1.0,6.0,0.0,3.0,1.0,0.0,0.0,0.0
8,"DOREY-HAVENS,AUBREY",0,5.0,0.0,5.0,1.0,2.0,4.0,5.0,3.0,5.0,3.0,1.0,4.0,0.0,1.0,2.0
9,"SIMPSON,NOAH",1,1.0,0.0,1.0,2.0,0.0,1.0,5.0,2.0,3.0,0.0,0.0,0.0,3.0,0.0,2.0


In [72]:
for index, row in lineup_event_df.iterrows():
    if index < 31:
        time_lineup = lineup_time_score_df.iloc[index]["lineup"].to_list()[0]
    else:
        time_lineup = []
    event_lineup = lineup_event_df.iloc[index]["lineup"]
    if time_lineup != event_lineup:
        print(event_lineup, time_lineup, index)

['BRAZDEIKIS,AUGUSTAS', 'DOREY-HAVENS,AUBREY', 'MILON,EMANUEL', 'SMITH,DANIEL', 'SPENCER,XAVIER'] [] 31
['BRAZDEIKIS,AUGUSTAS', 'DOREY-HAVENS,AUBREY', 'MILON,EMANUEL', 'OKADO,MARJOK', 'SMITH,DANIEL'] [] 32
['J-SERAPHIN,REGINALD', 'MILON,EMANUEL', 'OKADO,MARJOK', 'RANDRIASALAMA,TO', 'RILEY,BEN'] [] 33


off possession = made and miss

def possession = Turnover, defensive rebound,	Foul, Steal

efficiency = point, rebound, assist, steal, block, -missed free throw, -turnover, -points conceded

What type of game was it? (Pre-season, Regular Season, or Playoffs)

In [73]:
scoring_values = {'made layup': 2, 'made free throw': 1, 'made jump shot': 2, 'made 3-pt. jump shot': 3}
final_columns = ['Player Name', 'PtScored', 'ptsconceded', 
                #  'off possession', 'def possession', 
                "efficiency",
                'minutes', 'home/visitor', 'opponent', 'date', 'game_type']
player_final_table = pd.DataFrame(columns=[final_columns])
for index, row in event_num_df.iterrows():
    
    points_scored = 0
    for key in scoring_values:
        score_value = scoring_values[key]
        points_scored += event_num_df.loc[index, key] * score_value

    points_conceded = float(time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "score")])

    efficiency = sum([points_scored, row["offensive rebound"], row["defensive rebound"], row["Assist"], row["Steal"]]) - \
    sum([row["missed free throw"], row["Turnover"], points_conceded])
    efficiency = float(efficiency)

    minutes = time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "time")] / 60
    minutes = "{:.2f}".format(minutes.to_list()[0])
    hv_df = player_event_df.loc[player_event_df["player"] == row["player"]].iloc[1]
    if pd.isna(hv_df['Home']) == False:
        hv = "Home"
    else:
        hv = "Visitor"
    
    opponent_df = player_event_df.loc[pd.isna(player_event_df[hv]) == True]
    opponent = opponent_df.iloc[1]["Home"] if hv == "Visitor" else opponent_df.iloc[1]["Visitor"]

    new_row = {"Player Name":[row["player"]], "PtScored":[points_scored],
               "ptsconceded":[points_conceded], "efficiency":[efficiency], "minutes":minutes, 
               "home/visitor":hv, "opponent":opponent}
    
    new_df = pd.DataFrame(new_row)
    player_final_table = pd.concat([player_final_table, new_df], ignore_index=True, axis=0)

player_final_table = player_final_table.reindex(columns=final_columns)

/var/folders/hx/k_zr_cdj5l348hsyxvym51kr0000gn/T/ipykernel_4934/1449936464.py:14: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  points_conceded = float(time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "score")])
/var/folders/hx/k_zr_cdj5l348hsyxvym51kr0000gn/T/ipykernel_4934/1449936464.py:14: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  points_conceded = float(time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "score")])


In [74]:
player_final_table

,Player Name,PtScored,ptsconceded,efficiency,minutes,home/visitor,opponent,date,game_type
0,"MILON,EMANUEL",9.0,40.0,-27.0,27.75,Home,Canisius,NaN,NaN
1,"OKADO,MARJOK",9.0,41.0,-29.0,27.35,Home,Canisius,NaN,NaN
2,"RANDRIASALAMA,TO",2.0,16.0,-13.0,12.85,Home,Canisius,NaN,NaN
3,"RILEY,BEN",1.0,18.0,-19.0,11.97,Home,Canisius,NaN,NaN
4,"J-SERAPHIN,REGINALD",0.0,7.0,-7.0,10.07,Home,Canisius,NaN,NaN
5,"SPENCER,XAVIER",4.0,35.0,-26.0,17.45,Home,Canisius,NaN,NaN
6,"BRAZDEIKIS,AUGUSTAS",18.0,46.0,-24.0,25.37,Home,Canisius,NaN,NaN
7,"SMITH,DANIEL",9.0,53.0,-40.0,29.45,Home,Canisius,NaN,NaN
8,"DOREY-HAVENS,AUBREY",10.0,42.0,-29.0,23.55,Home,Canisius,NaN,NaN
9,"SIMPSON,NOAH",2.0,26.0,-20.0,12.25,Home,Canisius,NaN,NaN


## Let's go to lineup

In [75]:
if "Player Name" in final_columns:
    final_columns.remove("Player Name")
lineup_final_columns = ["Lineup"].extend(final_columns)
lineup_final_table = pd.DataFrame(columns=[lineup_final_columns])
for index, row in lineup_event_df.iloc[:-1].iterrows():
    
    points_scored = 0
    for key in scoring_values:
        score_value = scoring_values[key]
        points_scored += lineup_event_df.loc[index, key] * score_value

    points_conceded = float(lineup_time_score_df[("total", "score")].iloc[index])

    efficiency = sum([points_scored, row["offensive rebound"], row["defensive rebound"], row["Assist"], row["Steal"]]) - \
    sum([row["missed free throw"], row["Turnover"], points_conceded])
    efficiency = float(efficiency)

    minutes = lineup_time_score_df[("total", "time")].iloc[index] / 60
    minutes = "{:.2f}".format(minutes)

    hv_df = player_event_df.loc[player_event_df["player"] == row["lineup"][0]].iloc[1]
    if pd.isna(hv_df['Home']) == False:
        hv = "Home"
    else:
        hv = "Visitor"

    opponent_df = player_event_df.loc[pd.isna(player_event_df[hv]) == True]
    opponent = opponent_df.iloc[1]["Home"] if hv == "Visitor" else opponent_df.iloc[1]["Visitor"]

    new_row = {"Lineup":[row["lineup"]], "PtScored":[points_scored],
               "ptsconceded":[points_conceded], "efficiency":[efficiency], 
               "minutes":minutes, "home/visitor":hv, "opponent":opponent}
    
    new_df = pd.DataFrame(new_row)
    lineup_final_table = pd.concat([lineup_final_table, new_df], ignore_index=True, axis=0)

lineup_final_table = lineup_final_table.reindex(columns=lineup_final_columns)

In [76]:
lineup_final_table

,None,Lineup,PtScored,ptsconceded,efficiency,minutes,home/visitor,opponent
0,NaN,"[J-SERAPHIN,REGINALD, MILON,EMANUEL, OKADO,MARJOK, RANDRIASALAMA,TO, RILEY,BEN]",2.0,2.0,1.0,1.98,Home,Canisius
1,NaN,"[J-SERAPHIN,REGINALD, MILON,EMANUEL, OKADO,MARJOK, RILEY,BEN, SPENCER,XAVIER]",0.0,0.0,0.0,0.40,Home,Canisius
2,NaN,"[BRAZDEIKIS,AUGUSTAS, MILON,EMANUEL, OKADO,MARJOK, RILEY,BEN, SPENCER,XAVIER]",0.0,0.0,-1.0,0.33,Home,Canisius
3,NaN,"[BRAZDEIKIS,AUGUSTAS, MILON,EMANUEL, OKADO,MARJOK, SMITH,DANIEL, SPENCER,XAVIER]",0.0,0.0,0.0,0.25,Home,Canisius
4,NaN,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, MILON,EMANUEL, OKADO,MARJOK, SMITH,DANIEL]",5.0,5.0,3.0,1.58,Home,Canisius
5,NaN,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH, SMITH,DANIEL]",0.0,0.0,-1.0,0.95,Home,Canisius
6,NaN,"[DOREY-HAVENS,AUBREY, J-SERAPHIN,REGINALD, RANDRIASALAMA,TO, SIMPSON,NOAH, SMITH,DANIEL]",0.0,0.0,2.0,1.53,Home,Canisius
7,NaN,"[DOREY-HAVENS,AUBREY, RANDRIASALAMA,TO, RILEY,BEN, SIMPSON,NOAH, SMITH,DANIEL]",3.0,3.0,2.0,1.40,Home,Canisius
8,NaN,"[RANDRIASALAMA,TO, RILEY,BEN, SIMPSON,NOAH, SMITH,DANIEL, SPENCER,XAVIER]",2.0,4.0,1.0,1.22,Home,Canisius
9,NaN,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH, SMITH,DANIEL]",3.0,3.0,0.0,0.70,Home,Canisius


In [37]:
import pandas as pd

# Create a sample dataframe
data = {'A': [1, 2, 3], 'B': [4, 5, 6], "C":[4, 5, 6]}
df = pd.DataFrame(data)

# Create multi-level columns
df.columns = pd.MultiIndex.from_tuples([('Column 1', 'time'), ('Column 1', 'score'), ('Column 2', 'time')])

# Display the dataframe
print(df)

  Column 1       Column 2
      time score     time
0        1     4        4
1        2     5        5
2        3     6        6
